# **distilBert for PI redaction**

The following notebook contains the initial usage of the distilBERT model to evaluate whether or not it is a good performer for the project.

Step 1: imports

In [2]:
!pip install Trafilatura
from transformers import pipeline
from transformers import DistilBertTokenizer
from transformers import DistilBertForTokenClassification, Trainer, TrainingArguments
from trafilatura.core import *
from trafilatura import fetch_url
import re
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.2/132.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.4
    Uninstalling lxml-4.9.4:
      Successfully uninstalled lxml-4.9.4


Step 2: load model

In [3]:
# Load DistilBERT NER model
nlp = pipeline('ner', model='Davlan/distilbert-base-multilingual-cased-ner-hrl')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/539M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step 3: define function for PII detection and redaction

In [6]:
# Function to detect and redact personal information using Named Entity Recognition
def remove_personal_info(text):
    entities = nlp(text)
    pi_entities = ['B-PER', 'I-PER', 'EMAIL']
    redacted_text = text

    # Sort entities by position in the text to avoid overlapping replacements
    sorted_entities = sorted(entities, key=lambda x: x['start'], reverse=True)

    # Replace each entity text with '[REDACTED]'
    instances = 0
    for ent in sorted_entities:
        if ent['entity'] in pi_entities:
            redacted_text = redacted_text[:ent['start']] + '[REDACTED]' + redacted_text[ent['end']:]
            instances += 1

    return redacted_text, instances

Step 4: perform a Trafilatura extraction and PII removal on the ECE404 webpage as an example

In [7]:
# test a specific instance: the ECE 404 homepage
ece404html_content = fetch_url("https://engineering.purdue.edu/ece404/")
text = extract(ece404html_content, favor_recall=True) # favor_precision=True will cut out noise, favor_recall=True will keep more in
redacted_text, count = remove_personal_info(str(text))
print("Total instances of PI detected in ECE404 homepage: " + str(count))
print(str(redacted_text))

Total instances of PI detected in ECE404 homepage: 9
Instructor: [REDACTED][REDACTED] [REDACTED]k
-
Professor, ECE
E-mail: kak@purdue.edu (You must place the string 'ece404' in the subject line to get past your instructor's notorious spam filter)
Graduate TAs:
Ami[REDACTED] [REDACTED]shyap
-
E-mail: kashyap9 (at purdue dot edu)
[REDACTED] [REDACTED]
-
E-mail: wang3450 (at purdue dot edu)
[REDACTED] [REDACTED]
-
E-mail: dubois6 (at purdue dot edu)
Lecture Location and Time
-
TuTh: 6:00 PM - 7:15 PM, PHYS 112
Course Description
-
Beyond question, computer and network security has emerged as one of
the most important subjects of study in modern times. Even the minutest
details of our lives now depend on our computers and networks working
with our trust that the information that is private to us will not fall
in the hands of those with ill intent. The two major components of
computer and network security are cryptography and what is known as
systems-oriented security. For a good education 

Step 5: mount subset of GovDocs1 dataset from Google Drive for testing (this section of code is not reproducible unless you download GovDocs data yourself at this link https://corp.digitalcorpora.org/corpora/files/govdocs1/threads/ )

In [8]:
from google.colab import drive
drive.mount('/content/drive')
govdocs_dir ='/content/drive/My Drive/ECE570/govdocs_testingdata'

Mounted at /content/drive


Step 6: create a list of the html files from the dataset (again not reproducible without govdocs data downloaded locally, but there was no way to access these files without local download)

In [9]:
# Function to identify and read HTML files from the dataset
def find_html_files(directory):
    html_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has an HTML extension
            if file.endswith(".html") or file.endswith(".htm"):
                html_files.append(os.path.join(root, file))
    return html_files

# Find HTML files in the dataset
html_files = find_html_files(govdocs_dir)

Step 7: for each html file from GovDocs1 dataset, extract content with Trafilatura and use the model to remove PII (again not reproducible if you have not downloaded a GovDocs1 thread and saved it to your Drive)

In [10]:
# Process the HTML content with Trafilatura's extract() function
# detect and remove the PII with the distilBERT NER model
def process_html(html_content):
    text = extract(html_content, favor_recall=True)
    if text:
        # Apply PI redaction
        redacted_text, instances = remove_personal_info(text)
        return redacted_text, instances
    else:
        return None

# Function to read the content of an HTML file
def read_html_file(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        return f.read()

# for each HTML file, extract content and detect and remove PII
if html_files:
    i = 0
    total_instances = 0
    for file in html_files:
        html_content = read_html_file(file)
        redacted_content, instances = process_html(html_content)
        total_instances += instances
        if i == 13: # this was selected because it is an example of reasonably small enough length to display
            ex_for_print = redacted_content
        # if redacted_content:
        #     try:
        #         with open(f'./distilBERT_filtered_content/test{i}.txt', 'w', encoding='utf-8') as fp:
        #             fp.write(redacted_content)
        #     except Exception as e:
        #         print(f"Error writing to file: {e}")
        # else:
        #     print(f'Failed to extract text from {file}.')
        i += 1
    print("Total instances of PI detected in GovDocs subset: " + str(total_instances))
    print("Example: " + str(ex_for_print))
else:
    print("No HTML files found in the directory.")

Total instances of PI detected in GovDocs subset: 622
Example: Statement of robert lance [REDACTED][REDACTED], nominated by president bush to be a member of the board of trustees of the morris k. udall scholarship and excellence in national environmental policy foundation
To the united states senate committee on environment and public works
Submitted march 25, 2003
Mr. Chairman and Members of the Committee, thank you for the opportunity to provide this statement in support of my nomination to be a member of the Board of Trustees of the Morris K. Udall Foundation. I am honored and grateful that President [REDACTED] saw fit to nominate me to this position and, if confirmed, look forward to continuing my public service by helping to advance the mission of the Udall Foundation.
My professional career and personal background has provided me with valuable experience and perspective to bring to the Udall Foundation. I grew up in northern rural Michigan close to the shores of Lake Michigan. Fo

Step 8: download labeled dataset for accuracy testing

The dataset can be found here: https://huggingface.co/datasets/ai4privacy/pii-masking-400k

In [ ]:
!pip install datasets
from datasets import load_dataset
ds = load_dataset("ai4privacy/pii-masking-400k")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

1en.jsonl:   0%|          | 0.00/84.8M [00:00<?, ?B/s]

de.jsonl:   0%|          | 0.00/82.7M [00:00<?, ?B/s]

es.jsonl:   0%|          | 0.00/42.5M [00:00<?, ?B/s]

...

fr.jsonl:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

it.jsonl:   0%|          | 0.00/19.8M [00:00<?, ?B/s]

data/validation/nl.jsonl:   0%|          | 0.00/9.67M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/325517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/81379 [00:00<?, ? examples/s]

Step 9: Split the data, run through model, and compute accuracy based on location of redactions in the strings

In [ ]:
# select 1,000 rows from the validation set as no test set is available
data = ds['validation']
data = data[0:1000]
y_true = data['masked_text']
X = data['source_text']
y_pred = []
for x in X:
  y, _ = remove_personal_info(x)
  y_pred.append(y)

def normalize_text(text):
    text = text.replace(" ", "")
    text = text.replace("\n", "")
    return text.lower()

# finds each redaction and adds the redaction and the characters to its left and right into a tuple
# returns list of the tuples
def extract_redactions_with_context(text):
    redactions_with_context = []

    # Use regular expression to find all redactions and their positions
    for match in re.finditer(r'\[.*?\]', text):
        start = match.start()
        end = match.end()
        left_char = text[start - 1] if start > 0 else ''  # Get the left character
        right_char = text[end] if end < len(text) else ''  # Get the right character
        redactions_with_context.append((match.group(0), left_char, right_char))  # Store redaction and its context

    return redactions_with_context

# wrote own accuracy score function to compute accuracy of redactions based on what characters are to
# the left and right of each [redacted] segment
def compute_accuracy(y_true, y_pred):
    # normalize both true and predicted texts by removing all spaces, newlines, and converting to lowercase
    y_true_normalized = [normalize_text(str(t)) for t in y_true]
    y_pred_normalized = [normalize_text(str(p)) for p in y_pred]
    total_redactions = 0
    correct_matches = 0

    for true_text, pred_text in zip(y_true_normalized, y_pred_normalized):
      # find each redaction and the characters to its left and right, returns list of tuples
      true_redactions = extract_redactions_with_context(true_text)
      pred_redactions = extract_redactions_with_context(pred_text)

      # count total redactions in the true text
      total_redactions += len(pred_redactions)

      # check surrounding characters of predicted redactions against true redactions
      i = 0
      for pred_redaction, pred_left, pred_right in pred_redactions:
        if i < len(true_redactions) and true_redactions[i][1] == pred_left:
          correct_matches += 1
        if pred_right == '[': # the case when there are two back to back labels
            if i+1 < len(pred_redactions) and i < len(true_redactions) and pred_redactions[i+1][2] == true_redactions[i][2]:
              correct_matches += .5
        elif i < len(true_redactions) and true_redactions[i][2] == pred_right:
          correct_matches += .5
        i += 1

    # compute accuracy as the number of correct matches over the total redactions
    accuracy = correct_matches / total_redactions if total_redactions > 0 else 0
    return accuracy

# Calculate accuracy
accuracy = compute_accuracy(y_true, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 44.99%
